In [1]:
!(pip list | grep tqdm) || pip install tqdm
!(pip list | grep tensorflow-hub) || pip install tensorflow-hub
!(pip list | grep tokenizers) || pip install tokenizers

tqdm                     4.41.1         
tensorflow-hub           0.8.0          
     |████████████████████████████████| 3.8MB 7.8MB/s 


In [2]:
import os
import json

os.environ['TFHUB_DOWNLOAD_PROGRESS'] = '1'

from tqdm import tqdm
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from tokenizers import BertWordPieceTokenizer

In [4]:
!curl --output train.json https://code.aliyun.com/qhduan/dataset/raw/88b3182c9f9d6185935d4484dfefefc23f50eaa7/LCQMC/train.json
!curl --output dev.json https://code.aliyun.com/qhduan/dataset/raw/88b3182c9f9d6185935d4484dfefefc23f50eaa7/LCQMC/dev.json
!curl --output vocab.txt https://code.aliyun.com/qhduan/zh-bert/raw/0fb1d96ec2133fe25e66bee12fe387cbe1e52938/vocab.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.9M    0 25.9M    0     0  3014k      0 --:--:--  0:00:08 --:--:-- 6751k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1060k    0 1060k    0     0   328k      0 --:--:--  0:00:03 --:--:--  328k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  106k    0  106k    0     0  49297      0 --:--:--  0:00:02 --:--:-- 49297


In [5]:
train = [json.loads(x) for x in open('train.json')]
dev = [json.loads(x) for x in open('dev.json')]

In [6]:
print(train[0], dev[0])

{'sentence1': '喜欢打篮球的男生喜欢什么样的女生', 'sentence2': '爱打篮球的男生喜欢什么样的女生', 'label': '1'} {'sentence1': '开初婚未育证明怎么弄？', 'sentence2': '初婚未育情况证明怎么开？', 'label': '1'}


In [7]:
tokenizer = BertWordPieceTokenizer("vocab.txt")

In [12]:
def compose_data(data, batch_size=32):
    X = [
        tokenizer.encode(x.get('sentence1'), x.get('sentence2')).tokens
        for x in tqdm(data)
    ]
    Y = [int(x.get('label')) for x in data]
    X = tf.ragged.constant(X, tf.string)
    Y = tf.constant(Y, tf.int32)

    @tf.autograph.experimental.do_not_convert
    def _to_tensor(x, y):
        return x.to_tensor(), y

    return tf.data.Dataset.zip((
        tf.data.Dataset.from_tensor_slices(X),
        tf.data.Dataset.from_tensor_slices(Y)
    )).batch(batch_size).map(_to_tensor)

In [13]:
data_train = compose_data(train)
data_dev = compose_data(dev)


100%|██████████| 238766/238766 [00:10<00:00, 22683.67it/s]

100%|██████████| 8802/8802 [00:00<00:00, 22260.53it/s]


In [14]:
for x, y in data_train.take(1):
  print(x.shape, y.shape)

(32, 52) (32,)


In [15]:
bert = hub.KerasLayer(
    'https://code.aliyun.com/qhduan/zh-roberta-wwm/raw/2c0d7fd709e4719a9ab2ca297f51b24e20586dbe/zh-roberta-wwm-L12.tar.gz',
    output_key='sequence_output',
    trainable=False)

Downloaded https://code.aliyun.com/qhduan/zh-roberta-wwm/raw/2c0d7fd709e4719a9ab2ca297f51b24e20586dbe/zh-roberta-wwm-L12.tar.gz, Total size: 413.52MB



In [16]:
pred_y = bert(x)

In [17]:
print(pred_y.shape)

(32, 52, 768)


In [18]:
# 用tf.keras.Sequential的话，可能导致模型无法load
inputs = tf.keras.layers.Input(shape=(None,), dtype=tf.string)
m = inputs
m = bert(m)
m = tf.keras.layers.Masking()(m)
m = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128))(m)
m = tf.keras.layers.Dense(2, activation='softmax')(m)
model = tf.keras.Model(inputs=inputs, outputs=m)

In [19]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(1e-3),
    metrics=['acc']
)

In [20]:
model.predict(x)

array([[0.43755826, 0.5624417 ],
       [0.53252894, 0.4674711 ],
       [0.30023438, 0.6997656 ],
       [0.49681795, 0.5031821 ],
       [0.41369775, 0.5863023 ],
       [0.34808278, 0.6519173 ],
       [0.5030593 , 0.4969407 ],
       [0.5713579 , 0.4286421 ],
       [0.4890933 , 0.5109067 ],
       [0.4462429 , 0.5537571 ],
       [0.53797925, 0.46202073],
       [0.4028684 , 0.5971316 ],
       [0.4488191 , 0.5511809 ],
       [0.46879438, 0.53120565],
       [0.4438246 , 0.5561754 ],
       [0.47067833, 0.5293217 ],
       [0.40767556, 0.59232444],
       [0.4190747 , 0.5809253 ],
       [0.5082262 , 0.49177375],
       [0.4330195 , 0.5669805 ],
       [0.5793167 , 0.42068332],
       [0.34212926, 0.65787077],
       [0.31667903, 0.68332094],
       [0.49078268, 0.5092173 ],
       [0.42947218, 0.57052785],
       [0.45564246, 0.54435754],
       [0.53219545, 0.46780455],
       [0.5295534 , 0.47044662],
       [0.5102111 , 0.48978895],
       [0.44329828, 0.5567017 ],
       [0.

In [21]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, None, 768)         102880904 
_________________________________________________________________
masking (Masking)            (None, None, 768)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               918528    
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
Total params: 103,799,946
Trainable params: 919,042
Non-trainable params: 102,880,904
_________________________________________________________________


In [22]:
model.fit(data_train, epochs=1, validation_data=data_dev)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


7462/7462 [==============================] - 740s 99ms/step - loss: 0.2657 - acc: 0.8876 - val_loss: 0.3993 - val_acc: 0.8154


In [23]:
model.evaluate(data_dev)

276/276 [==============================] - 22s 81ms/step - loss: 0.3993 - acc: 0.8154


[0.3993489146232605, 0.8153828382492065]

In [24]:
model.save('./test_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: ./test_model/assets


INFO:tensorflow:Assets written to: ./test_model/assets


In [25]:
tf.keras.models.load_model('./test_model')